In [ ]:
import numpy as np
import scipy as scp
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.special as scps
from statsmodels.graphics.gofplots import qqplot
from scipy.linalg import cholesky
from functools import partial
from functions.probabilities import Heston_pdf, Q1, Q2
from functions.cython.cython_Heston import Heston_paths_log, Heston_paths
from scipy.optimize import minimize

from IPython.display import display
import sympy; sympy.init_printing()

def display_matrix(m):
    display(sympy.Matrix(m))

SIZE = 1000000
rho = 0.6

MU = np.array([0, 0])
COV = np.array( [[1, rho], [rho, 1]] ); 
print("COV: "); display_matrix(COV)

Y = ss.multivariate_normal.rvs( mean=MU, cov=COV, size=SIZE )

print("correlation matrix: "); display_matrix(np.corrcoef(Y.T).round(2))

Y_1 = np.random.normal(loc=0, scale=1, size=SIZE )
Y_2 = rho * Y_1 + np.sqrt(1-rho**2) * np.random.normal(loc=0, scale=1, size=SIZE )
print("correlation matrix: "); display_matrix(np.corrcoef(Y_1,Y_2).round(2))


print("Cholesky: "); display_matrix( cholesky(COV) )
print("Correlation matrix: "); display_matrix( cholesky(COV, lower=True) @ cholesky(COV) )

%%time
np.random.seed(seed=42) 

N = 1000000             # time steps 
paths = 3               # number of paths
T = 1
T_vec, dt = np.linspace(0,T,N, retstep=True )
dt_sq = np.sqrt(dt)

S0 = 100          # spot price
X0 = np.log(S0)   # log price
v0 = 0.04         # spot variance
Y0 = np.log(v0)   # log-variance 

mu = 0.1                                           # drift
rho = -0.2                                         # correlation coefficient
kappa = 2                                          # mean reversion coefficient
theta = 0.04                                       # long-term variance
sigma = 0.3                                        # Vol of Vol - Volatility of instantaneous variance
std_asy = np.sqrt( theta * sigma**2 /(2*kappa) )   # asymptotic standard deviation for the CIR process
assert(2*kappa * theta > sigma**2)                 # Feller condition

# Generate random Brownian Motion
MU = np.array([0, 0])
COV = np.matrix([[1, rho], [rho, 1]])
W = ss.multivariate_normal.rvs( mean=MU, cov=COV, size=(paths,N-1) )
W_S = W[:,:,0]   # Stock Brownian motion:     W_1
W_v = W[:,:,1]   # Variance Brownian motion:  W_2

# Initialize vectors
Y = np.zeros((paths,N))
Y[:,0] = Y0
X = np.zeros((paths,N))
X[:,0] = X0
v = np.zeros(N)

# Generate paths
for t in range(0,N-1):
    v = np.exp(Y[:,t])    # variance 
    v_sq = np.sqrt(v)     # square root of variance 
    
    Y[:,t+1] = Y[:,t] + (1/v)*( kappa*(theta - v) - 0.5*sigma**2 )*dt + sigma * (1/v_sq) * dt_sq * W_v[:,t]   
    X[:,t+1] = X[:,t] + (mu - 0.5*v)*dt + v_sq * dt_sq * W_S[:,t]
    
fig = plt.figure(figsize=(16,5))
ax1 = fig.add_subplot(121); ax2 = fig.add_subplot(122)

ax1.plot(T_vec, np.exp(X.T) )
ax1.set_title("Heston model, Stock process. 3 paths"); ax1.set_xlabel("Time"); ax1.set_ylabel("Stock")
ax2.plot(T_vec, np.exp(Y.T) )
ax2.set_title("Heston model, Variance process. 3 paths"); ax2.set_xlabel("Time"); ax2.set_ylabel("Variance")
ax2.plot(T_vec, (theta + std_asy)*np.ones_like(T_vec), label="1 asymptotic std dev", color="black" )
ax2.plot(T_vec, (theta - std_asy)*np.ones_like(T_vec), color="black" )
ax2.plot(T_vec, theta*np.ones_like(T_vec), label="Long term mean" )
ax2.legend(loc="upper right"); 
plt.show()


mu = 0.1                                           # drift
rho = -0.9                                         # correlation coefficient
kappa = 2                                          # mean reversion coefficient
theta = 0.04                                       # long-term mean of the variance
sigma = 0.3                                        # (Vol of Vol) - Volatility of instantaneous variance
T = 1                                              # Terminal time
v0 = 0.04                                          # spot variance
S0 = 100                                           # spot stock price 

c = 2*kappa / ((1-np.exp(-kappa*T))*sigma**2)
df = 4*kappa*theta / sigma**2                      # degrees of freedom
nc = 2*c*v0*np.exp(-kappa*T)                       # non-centrality parameter 

_,_ = Heston_paths_log(N=500, paths=20000, T=T, S0=S0, v0=v0, 
                  mu=mu, rho=rho, kappa=kappa, theta=theta, sigma=sigma  )

%%time
S,V = Heston_paths(N=20000, paths=20000, T=T, S0=S0, v0=v0, 
                  mu=mu, rho=rho, kappa=kappa, theta=theta, sigma=sigma  )

log_R = np.log(S/S0)
x = np.linspace(log_R.min(), log_R.max(), 500)
y = np.linspace(0.00001, 0.3, 500)

fig = plt.figure(figsize=(16,5))
ax1 = fig.add_subplot(121); ax2 = fig.add_subplot(122)

ax1.plot(y, ss.ncx2.pdf(y, df, nc, scale=1/(2*c)), color="green", label="non-central-chi-squared")
ax1.hist(V, density=True, bins=100, facecolor="LightBlue", label="frequencies of v_T")
ax1.legend(); ax1.set_title("Histogram vs CIR distribution"); ax1.set_xlabel("v_T")

ax2.plot(x, ss.norm.pdf(x, log_R.mean(), log_R.std(ddof=0)), color='r', label="Normal density")
ax2.hist(log_R, density=True, bins=100, facecolor="LightBlue", label="frequencies of log-returns")
ax2.legend(); ax2.set_title("Histogram vs log-returns distribution"); ax2.set_xlabel("log(S_T/S0)")
plt.show()

qqplot(log_R, line='s');  plt.show()

%%time
S2,_ = Heston_paths(N=20000, paths=20000, T=T, S0=S0, v0=v0, 
                  mu=mu, rho=0.9, kappa=kappa, theta=theta, sigma=sigma  )
log_R2 = np.log(S2/S0)

x = np.linspace(-1, 1, 70)

fig = plt.figure(figsize=(16,5))
ax1 = fig.add_subplot(121); ax2 = fig.add_subplot(122)

ax1.plot(x, [Heston_pdf(i, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=0.9) for i in x], 
       color='blue', label="Heston density" )
ax1.plot(x, ss.norm.pdf(x, log_R2.mean(), log_R2.std(ddof=0)), color='r', label="Normal density")
ax1.hist(log_R2, density=True, bins=100, facecolor="LightBlue", label="frequencies of R_T")
ax1.legend(); ax1.set_title("Histogram vs log-returns, Positive skew"); ax1.set_xlabel("log(S_T/S0)")

ax2.plot(x, [Heston_pdf(i, t=T, v0=v0, mu=mu, theta=theta, sigma=sigma, kappa=kappa, rho=-0.9) for i in x], 
       color='blue', label="Heston density" )
ax2.plot(x, ss.norm.pdf(x, log_R.mean(), log_R.std(ddof=0)), color='r', label="Normal density")
ax2.hist(log_R, density=True, bins=100, facecolor="LightBlue", label="frequencies of R_T")
ax2.legend(loc="upper left") 
ax2.set_title("Histogram vs log-returns, Negative skew"); ax2.set_xlabel("log(S_T/S0)")
plt.show()

print("The skewness for rho= -0.9 is: ", ss.skew(log_R) )
print("The skewness for rho= +0.9 is: ", ss.skew(log_R2) )

